##Step 1 : Import Module

In [1]:
import numpy as np
from enum import Enum, auto
from IPython.core.display import HTML,display
import pandas as pd
import os
from datetime import datetime

##Step 2 : Deklarasi Variabel

In [2]:
# String Variable
sequenceA       : str = 'FEEDS'#GGGAACCCAT'#'TREES'
sequenceB       : str = 'TREES'#ATAGAAATGA'#'FEEDS'
alignedA        : str
alignedB        : str

# Integer Variable
lengthSequenceA : int
lengthSequenceB : int

limit = 99999

# Matrix
main_matrix           : list
match_checker_matrix  : list

In [3]:
class Sequences(Enum) :
  MATCH: int = 1
  MISS_MATCH = -1
  GAP_PENALTY = -1

class Arrow(Enum) :
  UP          : str = "\u2191"
  RIGHT       : str = "\u2192"
  down_arrow  : str = "\u2193"
  LEFT        : str = "\u2190"
  DOWN_RIGHT  : str = "\u2198"
  UP_LEFT     : str = "\u2196"

##Step 3 : Deklarasi Method Function

In [4]:
def reverseFileLine(fileName: str):
  r = open(fileName+ ".txt", "r")
  f = open("result1.txt", "w")
  f2 = open("result2.txt", "w")

  test = r.readline()
  # print(test)

  first = True
  loop = 0
  index = 0

  while True:
    # print(index)
    index = index + 1
    if (test != ""):
      # print("=======")
      if (loop == 0):
        if (first):
          # print("first : " + test)
          f.write(test)
          f.close()
          first = False
        else:
          f = open("result1.txt", "w")
          # print("if 0 : " + test)

          f.write(test)
          f.close()

          # !cat result2.txt
          !cat result2.txt >> result1.txt
          # !echo "" > result2.txt
          pass
        loop = 1
      else:
        # print("else : " + test)
        f2 = open("result2.txt", "w")
        f2.write(test)
        f2.close()

        # !cat result1.txt
        !cat result1.txt >> result2.txt
        # !echo "" > result1.txt

        loop = 0
    else:
      break
    test = r.readline()


  if (loop == 0):
    !rm result1.txt
    !mv result2.txt result.txt
  else:
    !rm result2.txt
    !mv result1.txt result.txt

In [5]:
def parseDataText(dataList: list, initial: str):
  f = open(initial+"_buffer"+".txt", "a")
  data = ""
  for i in range(0, len(dataList)):
    char = str(dataList[i])
    # print(char)
    if i == len(dataList) -1:
      data=data+","+char
    else:
      if (i == 0):
        data=char
      else:
        data=data+","+char

    # print(data)
  f.write(data+"\n")
  f.close()

In [6]:
def buildCacheArray(lenAtas : int, lenKiri : int, char : str, initial: str):
  data = ""
  filename = initial+".txt"
  if (os.path.exists(filename)):
    os.remove(filename)
  f = open(filename, "a")
  for i in range(0, lenAtas):
    # print(i)
    if i == lenAtas -1:
      data=data+char
    else:
      data=char+","+data

  for i in range(0, lenKiri):
    f.write(data+"\n")

In [7]:
# Fungsi generate data table
def pretty_table_from_array(data_array, row_labels,col_labels):
    """Show an HTML table from a 2d numpy array"""
    df = pd.DataFrame(data_array,index=row_labels,columns=col_labels)
    table_html = df.to_html()
    return HTML(table_html)

In [8]:
def traceback_alignment_cache(fileName: str, sequenceA, sequenceB):
    fileTraceback = open(fileName, "r")

    seqData = fileTraceback.readline().split(",")
    seqData[len(seqData) - 1] = seqData[len(seqData) - 1].replace('\n', '')

    row = len(sequenceA)
    col = len(sequenceB)
    arrow = seqData[col]
    aligned_sequenceA = ""
    aligned_sequenceB = ""
    alignment_indicator = ""
    # print(sequenceA)
    # print(sequenceB)
    while arrow != "-":
            # print("Currently on row:",row)
            # print("Currently on col:",col)
            arrow = seqData[col]
            # print("Arrow:",arrow)

            if arrow == Arrow.UP.value:
                # print("insert indel into top sequence")
                #We want to add the new indel onto the left
                #side of the growing aligned sequence
                aligned_sequenceB = "-"+aligned_sequenceB
                aligned_sequenceA = sequenceA[row-1] + aligned_sequenceA
                alignment_indicator = " "+alignment_indicator
                row -=1
                seqData = fileTraceback.readline().split(",")
                seqData[len(seqData) - 1] = seqData[len(seqData) - 1].replace('\n', '')

            elif arrow == Arrow.UP_LEFT.value:
                # print("match or mismatch")
                #Note that we look up the row-1 and col-1 indexes
                #because there is an extra "-" character at the
                #start of each sequence
                sequenceA_character = sequenceA[row-1]
                sequenceB_character = sequenceB[col-1]
                aligned_sequenceA = sequenceA[row-1] + aligned_sequenceA
                aligned_sequenceB = sequenceB[col-1] + aligned_sequenceB
                if sequenceA_character == sequenceB_character:
                    alignment_indicator = "|"+alignment_indicator
                else:
                    alignment_indicator = " "+alignment_indicator
                row -=1
                col -=1
                seqData = fileTraceback.readline().split(",")
                seqData[len(seqData) - 1] = seqData[len(seqData) - 1].replace('\n', '')

            elif arrow == Arrow.LEFT.value:
                # print("Insert indel into left sequence")
                aligned_sequenceA = "-"+aligned_sequenceA
                aligned_sequenceB = sequenceB[col-1] + aligned_sequenceB
                alignment_indicator = " "+alignment_indicator
                col -=1

            elif arrow == "-":
                break
            else:
                raise ValueError(f"Traceback array entry at {row},{col}: {arrow} is not recognized as an up arrow ({Arrow.UP.value}),Arrow.LEFT.value ({Arrow.LEFT.value}), Arrow.UP_LEFT.value ({Arrow.UP_LEFT.value}), or a stop (-).")
            #print(traceback_array,-row,-col,traceback_array[-row,-col])
    print(aligned_sequenceA)
    print(alignment_indicator)
    print(aligned_sequenceB)

    return aligned_sequenceA,aligned_sequenceB

In [9]:
def traceback_alignment(traceback_array, sequenceA, sequenceB):

    n_rows = len(sequenceA) + 1 #need an extra row up top
    n_columns = len(sequenceB) + 1 #need an extra row up top

    row = len(sequenceA)
    col = len(sequenceB)
    arrow = traceback_array[row,col]
    aligned_sequenceA = ""
    aligned_sequenceB = ""
    alignment_indicator = ""
    while arrow != "-":
            # print("Currently on row:",row)
            # print("Currently on col:",col)
            arrow = traceback_array[row,col]
            # print("Arrow:",arrow)

            if arrow == Arrow.UP.value:
                # print("insert indel into top sequence")
                #We want to add the new indel onto the left
                #side of the growing aligned sequence
                aligned_sequenceB = "-"+aligned_sequenceB
                aligned_sequenceA = sequenceA[row-1] + aligned_sequenceA
                alignment_indicator = " "+alignment_indicator
                row -=1

            elif arrow == Arrow.UP_LEFT.value:
                # print("match or mismatch")
                #Note that we look up the row-1 and col-1 indexes
                #because there is an extra "-" character at the
                #start of each sequence
                sequenceA_character = sequenceA[row-1]
                sequenceB_character = sequenceB[col-1]
                aligned_sequenceA = sequenceA[row-1] + aligned_sequenceA
                aligned_sequenceB = sequenceB[col-1] + aligned_sequenceB
                if sequenceA_character == sequenceB_character:
                    alignment_indicator = "|"+alignment_indicator
                else:
                    alignment_indicator = " "+alignment_indicator
                row -=1
                col -=1

            elif arrow == Arrow.LEFT.value:
                # print("Insert indel into left sequence")
                aligned_sequenceA = "-"+aligned_sequenceA
                aligned_sequenceB = sequenceB[col-1] + aligned_sequenceB
                alignment_indicator = " "+alignment_indicator
                col -=1

            elif arrow == "-":
                break
            else:
                raise ValueError(f"Traceback array entry at {row},{col}: {arrow} is not recognized as an up arrow ({Arrow.UP.value}),Arrow.LEFT.value ({Arrow.LEFT.value}), Arrow.UP_LEFT.value ({Arrow.UP_LEFT.value}), or a stop (-).")
            #print(traceback_array,-row,-col,traceback_array[-row,-col])
    print(aligned_sequenceA)
    print(alignment_indicator)
    print(aligned_sequenceB)

    return aligned_sequenceA,aligned_sequenceB

In [10]:
def buildCacheArrayWithRowNuberChar(lenAtas : int, lenKiri : int, char : str, initial: str, traceback: bool, initNumberRow : int = 0, cekpoint: bool = False):
  data = ""
  filename = initial+".txt"
  if (os.path.exists(filename)):
    os.remove(filename)
  f = open(filename, "a")
  for i in range(initNumberRow, lenKiri):
    if (i == initNumberRow):
      if (traceback):
        data="-,"+((lenAtas-2)*(Arrow.LEFT.value+","))+Arrow.LEFT.value
      else:
        if cekpoint:
          file = open("cekpoint.txt", "r")
          print(data)
          data = file.readline()
          file.close()
        else:
          for i in range(0, lenAtas):
            # print(i)
            if i == 0:
              data = "0"
            else:
              data=data+","+str(i*-1)
    else:
      if (traceback):
        data=Arrow.UP.value+","+((lenAtas-2)*(char+","))+char
      else:
        data=str((i*-1))+","+((lenAtas-2)*(char+","))+char
    f.write(data+"\n")

##Step 4 : Matrix initiate

In [11]:
buildCacheArray(len(sequenceA) + 1, len(sequenceB) + 1, "0", "main_matriks")
buildCacheArray(len(sequenceA) + 1, len(sequenceB) + 1, "-", "traceback_matriks")

##Step 5 : Needleman Wunsch Algorithm With Cache

### Step 1 : Initialisation

#### Old ALgorithm

In [12]:
mainData = open("main_matriks.txt", "r")
tracebackData = open("traceback_matriks.txt", "r")

In [13]:
if (os.path.exists("main_matriks_buffer.txt")):
  os.remove("main_matriks_buffer.txt")
mainData = open("main_matriks.txt", "r")
line = mainData.readline().split(",")
line[len(line) - 1] = line[len(line) - 1].replace('\n', '')
# print(line)
i = -1

while True:
  i = i + 1
  if i == 0:
    # print("if")
    for j in range(0, len(line)):
      line[j] = int(j) * Sequences.GAP_PENALTY.value
    parseDataText(line, "main_matriks")
    # print(line)
  else:
    # print("else")
    line = mainData.readline()
    # print(line)
    if len(line) != 0:
      line = line.split(",")
      line[len(line) - 1] = line[len(line) - 1].replace('\n', '')
      line[0] = i * Sequences.GAP_PENALTY.value
      parseDataText(line, "main_matriks")
    else:
      break

!rm main_matriks.txt
!mv main_matriks_buffer.txt main_matriks.txt

In [14]:
if (os.path.exists("traceback_matriks_buffer.txt")):
  os.remove("traceback_matriks_buffer.txt")
mainData = open("traceback_matriks.txt", "r")
line = mainData.readline().split(",")
line[len(line) - 1] = line[len(line) - 1].replace('\n', '')
# print(line)
i = -1

# 1 = kanan
# 2 = kiri
# 3 = atas
# 4 = bawah

while True:
  i = i + 1
  print(i)
  if i == 0:
    # print("if")
    for j in range(0, len(line)):
      if (j != 0):
        line[j] = Arrow.LEFT.value
    parseDataText(line, "traceback_matriks")
    # print(line)
  else:
    # print("else")
    line = mainData.readline()
    # print(line)
    if len(line) != 0:
      line = line.split(",")
      line[len(line) - 1] = line[len(line) - 1].replace('\n', '')
      line[0] = Arrow.UP.value
      parseDataText(line, "traceback_matriks")
    else:
      break

mainData.close()
tracebackData.close()

!rm traceback_matriks.txt
!mv traceback_matriks_buffer.txt traceback_matriks.txt

0
1
2
3
4
5
6


#### New Algorithm

In [15]:
start = datetime.now()
buildCacheArrayWithRowNuberChar(len(sequenceA) + 1, len(sequenceB) + 1, "0", "main_matriks", False)
buildCacheArrayWithRowNuberChar(len(sequenceA) + 1, len(sequenceB) + 1, "-", "traceback_matriks", True)
print("time :: " + str(datetime.now() - start))

time :: 0:00:00.001280


In [16]:
# 1 Menit 40 Detik
# print(len(sequenceA))
start = datetime.now()
buildCacheArrayWithRowNuberChar(len(sequenceA) + 1, len(sequenceB) + 1, "0", "main_matriks", False)
print("time :: " + str(datetime.now() - start))

time :: 0:00:00.001052


In [17]:
# 1 Menit 40 Detik
start = datetime.now()
buildCacheArrayWithRowNuberChar(len(sequenceA) + 1, len(sequenceB) + 1, "-", "traceback_matriks", True)
print("time :: " + str(datetime.now() - start))

time :: 0:00:00.000456


### Step 2 : Matrix Filling

In [18]:
mainData = open("main_matriks.txt", "r")
tracebackData = open("traceback_matriks.txt", "r")

if (os.path.exists("traceback_matriks_buffer.txt")):
  os.remove('traceback_matriks_buffer.txt')

if (os.path.exists("main_matriks_buffer.txt")):
  os.remove('main_matriks_buffer.txt')

lineRead = mainData.readline()
line1 = lineRead.split(",")
line1[len(line1) - 1] = line1[len(line1) - 1].replace('\n', '')
parseDataText(line1, "main_matriks")

lineRead = mainData.readline()
line2 = lineRead.split(",")
line2[len(line2) - 1] = line2[len(line2) - 1].replace('\n', '')

lineRead = mainData.readline()

treaceDataRead = tracebackData.readline()
lineTrace = treaceDataRead.split(",")
lineTrace[len(lineTrace) - 1] = lineTrace[len(lineTrace) - 1].replace('\n', '')
parseDataText(lineTrace, "traceback_matriks")

treaceDataRead = tracebackData.readline()

i = 1
while True:
  print
  if (lineRead != ""):
    line3 = lineRead.split(",")
    line3[len(line3) - 1] = line3[len(line3) - 1].replace('\n', '')

    lineTrace = treaceDataRead.split(",")
    lineTrace[len(lineTrace) - 1] = lineTrace[len(lineTrace) - 1].replace('\n', '')
  else:
    line3 = []

  # print("=======" + str(i))
  # print(line1)
  # print(line2)

  for j in range(1,len(sequenceA)+1):
    cell_to_the_left = int(line2[j-1])
    from_left_score = cell_to_the_left + Sequences.GAP_PENALTY.value

    above_cell = int(line1[j])
    from_above_score = above_cell + Sequences.GAP_PENALTY.value

    diagonal_left_cell = int(line1[j-1])

    if sequenceB[i-1] == sequenceA[j-1]:
        diagonal_left_cell_score = diagonal_left_cell + Sequences.MATCH.value
    else:
        diagonal_left_cell_score = diagonal_left_cell + Sequences.MISS_MATCH.value

    score = max([from_left_score,from_above_score,diagonal_left_cell_score])

    # main_matrix[i][j] = score
    line2[j] = score

    if score == from_left_score:
        arrow = Arrow.LEFT.value
    elif score == from_above_score:
        arrow = Arrow.UP.value
    elif score == diagonal_left_cell_score:
        arrow = Arrow.UP_LEFT.value
    lineTrace[j] = arrow
    # match_checker_matrix[i][j] = arrow

  # print("hasil : ")
  # print(line2)

  parseDataText(line2, "main_matriks")
  parseDataText(lineTrace, "traceback_matriks")

  line1 = line2
  line2 = line3
  lineRead = mainData.readline()
  treaceDataRead = tracebackData.readline()


  i = i + 1

  if lineRead == '' and len(line2) == 0:
    break

# for i in range(0, len(main_matrix)):

# print("Scoring array:")
# display(pretty_table_from_array(main_matrix,row_labels,column_labels))
# mainFunction()

### Step 3 : Traceback

In [19]:
!cat traceback_matriks_buffer.txt

-,←,←,←,←,←
↑,↖,←,←,←,←
↑,↑,↖,←,←,←
↑,↑,↖,↖,←,←
↑,↑,↑,↖,←,←
↑,↑,↑,↑,↖,↖


In [20]:
reverseFileLine("traceback_matriks_buffer")
traceback_alignment_cache("result.txt", sequenceB, sequenceA)

TREE-S
  || |
F-EEDS


('TREE-S', 'F-EEDS')